# Installations and Import Packages

In [1]:
print('Installations')
#!pip install geopandas
#!pip install contextily

Installations


In [4]:
import math
import time                       # Track loading times
import requests                   # Make HTTP requests
import json                       # Load US Census datasets as JSON
import chardet                    # Character encoding auto-detection
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Build Dataset For Spectral Clustering
### Load PUMA/CBSA(MSA) Crosswalk and List of Top 50 MSAs by Population

In [6]:
# Load PumaCBSACrosswalk.csv file (Use CBSA as MSA)

with open('PumaCBSACrosswalk.csv', 'rb') as f:
    ec = chardet.detect(f.read())                           # Detect encoding of the file (use readline() if file is large)
pumaMSACrossDF = pd.read_csv('PumaCBSACrosswalk.csv', encoding=ec['encoding'])
pumaMSACrossDF.columns = ['ST', 'PUMA', 'CBSA#', 'stab', 'CBSA']  # Format column names
pumaMSACrossDF['ST'] = pumaMSACrossDF['ST'].astype('string')    # Convert 'ST' (State) column to string type
print('\033[1m' + 'PumaCBSACrosswalk.csv') 
print('\033[0m' + '5 Features, 2203 Entries')
pumaMSACrossDF.head()

PumaCBSACrosswalk.csv
5 Features, 2203 Entries


,ST,PUMA,CBSA#,stab,CBSA
0,1,100,22520,AL,"Florence-Muscle Shoals, AL"
1,1,200,26620,AL,"Huntsville, AL"
2,1,301,26620,AL,"Huntsville, AL"
3,1,302,26620,AL,"Huntsville, AL"
4,1,400,22840,AL,"Fort Payne, AL"


In [7]:
# Create list for top 50 MSAs by population - load excel file

MSA50 = list(pd.read_excel('Top50MSAByPop.xlsx', sheet_name='Sheet1').iloc[3:,1])

# Remove unnecessary punctuation and suffixes
for i in range(len(MSA50)):
  MSA50[i] = MSA50[i].removeprefix('.')
  MSA50[i] = MSA50[i].removesuffix(' Metro Area')

print('\033[1m' + 'Top50MSAByPop.xlsx')
print('\033[0m' + '50 Entries')
print('Top 5:')
MSA50[:5]

Top50MSAByPop.xlsx
50 Entries
Top 5:


['New York-Newark-Jersey City, NY-NJ-PA',
 'Los Angeles-Long Beach-Anaheim, CA',
 'Chicago-Naperville-Elgin, IL-IN-WI',
 'Dallas-Fort Worth-Arlington, TX',
 'Houston-The Woodlands-Sugar Land, TX']

### Extract Yearly ACS Housing and Population Data from api.census.gov
### Merge 2012 - 2019 ACS Datasets

In [8]:
def readACS(year):
  ''' Input: year
      Output: Top 50 MSA's for the given year - csv file
      Return: Top 50 MSA's for the given year - pandas dataframe
  '''

  # Extract housing information from US Census PUMS dataset of given year
  # If year = 2021, extract 'YRBLT' instead of 'YBL'
  if year != 2021:
    url1 = 'https://api.census.gov/data/'+str(year)+'/acs/acs1/pums?get=SERIALNO,WGTP,GRPIP,RNTP,HINCP,TEN,OCPIP,VACS,YBL,BLD,ST,PUMA'
  else:
    url1 = 'https://api.census.gov/data/'+str(year)+'/acs/acs1/pums?get=SERIALNO,WGTP,GRPIP,RNTP,HINCP,TEN,OCPIP,VACS,YRBLT,BLD,ST,PUMA'
  response1 = requests.get(url1)
  if response1.status_code == 200:             # HTTP response status code 200 means success
    print('Housing file retrieved.')
  census1 = response1.text                     # Return the content of the response in unicode
  census1 = json.loads(census1)                # Parse a valid JSON string and convert it into a Python Dictionary
  hous = pd.DataFrame.from_dict(census1)       # Construct pandas DataFrame from dictionary
  columns = hous.iloc[0,:]
  hous.columns = columns
  hous.drop(index=0, inplace=True)

  # Extract attributes from population files
  url1 = 'https://api.census.gov/data/' + str(year) + '/acs/acs1/pums?get=AGEP,SERIALNO,RAC1P,SCHL,HISP,ESR&SPORDER=1'
  response1 = requests.get(url1)
  if response1.status_code == 200:
    print('\nPopulation file retrieved.')
  census1 = response1.text
  census1 = json.loads(census1)
  pop = pd.DataFrame.from_dict(census1)
  columns = pop.iloc[0,:]
  pop.columns = columns
  pop.drop(index=0, inplace=True)

  # Merge housing and population dataframes - Left join on SERIALNO
  print('\nMerging housing and population datasets.')
  df1 = pd.merge(hous,pop,on = 'SERIALNO',how = 'left')

  # Merge with PUMA crosswalk file (PumaCBSACrosswalk.csv) - join on ST and PUMA
  print('\nAdding the MSAs using crosswalk file.')
  df_merge = pd.merge(df1,pumaMSACrossDF,how ='left', right_on=['ST','PUMA'], left_on=['ST','PUMA'])

  #Filtering the dataframe based on the MSAs
  print('\nFiltering required MSAs')
  msa50df = df_merge[df_merge['CBSA'].isin(MSA50)]
  msa50df.index = range(msa50df.shape[0])

  # Add year attribute
  msa50df.loc[:,'Year'] = year

  # Write dataframe to drive as csv file
  print('\nWriting into CSV file.')
  msa50df.to_csv(r'acs'+str(year)+'.csv')
  print('\nDone.')

  return msa50df

In [19]:
# Extract acs datasets with US Census Bureau API and export as csv files
# Approximately between 2.5 and 3.5 minutes per year

#acs2012 = readACS(2012)
#acs2013 = readACS(2013)
#acs2014 = readACS(2014)
#acs2015 = readACS(2015)
#acs2016 = readACS(2016)
#acs2017 = readACS(2017)
#acs2018 = readACS(2018)
#acs2019 = readACS(2019)
#acs2021 = readACS(2021)

# Save Time if csv files have already been extracted
# Load datasets as csv files and use first column as index

#acs2012 = pd.read_csv('acs2012.csv', index_col = 0)
#acs2013 = pd.read_csv('acs2013.csv', index_col = 0)
#acs2014 = pd.read_csv('acs2014.csv', index_col = 0)
#acs2015 = pd.read_csv('acs2015.csv', index_col = 0)
#acs2016 = pd.read_csv('acs2016.csv', index_col = 0)
#acs2017 = pd.read_csv('acs2017.csv', index_col = 0)
#acs2018 = pd.read_csv('acs2018.csv', index_col = 0)
acs2019 = pd.read_csv('acs2019.csv', index_col = 0)
acs2021 = pd.read_csv('acs2021.csv', index_col = 0)

In [22]:
acs2019.isna().sum()

SERIALNO        0
WGTP            0
GRPIP           0
RNTP            0
HINCP           0
TEN             0
OCPIP           0
VACS            0
YBL             0
BLD             0
ST              0
PUMA            0
AGEP        40307
RAC1P       40307
SCHL        40307
HISP        40307
ESR         40307
SPORDER     40307
CBSA#           0
stab            0
CBSA            0
Year            0
dtype: int64

In [16]:
acs2021.isna().sum()

SERIALNO       0
WGTP           0
GRPIP          0
RNTP           0
HINCP          0
TEN            0
OCPIP          0
VACS           0
YRBLT          0
BLD            0
ST             0
PUMA           0
AGEP        3012
RAC1P       3012
SCHL        3012
HISP        3012
ESR         3012
SPORDER     3012
CBSA#          0
stab           0
CBSA           0
Year           0
dtype: int64

In [8]:
# Merge all acs files
acsTotal = pd.concat([acs2012, acs2013, acs2014, acs2015, acs2016, acs2017, acs2018, acs2019], ignore_index=True) # no ACS2021

# Confirm the total length
totalLength = acs2012.shape[0] + acs2013.shape[0] + acs2014.shape[0] + acs2015.shape[0] + acs2016.shape[0] + acs2017.shape[0] + \
acs2018.shape[0] + acs2019.shape[0] # + acs2021.shape[0]
print('\033[1m' + 'Total Housing/Population data')
print('\033[0m' + 'Total length should be: {}'.format(totalLength))
print('{} Features, {} Entries'.format(acsTotal.shape[1], acsTotal.shape[0]))


acsTotal.head()


Total Housing/Population data
Total length should be: 5827854
22 Features, 5827854 Entries


,SERIALNO,WGTP,GRPIP,RNTP,HINCP,TEN,OCPIP,VACS,YBL,BLD,...,AGEP,RAC1P,SCHL,HISP,ESR,SPORDER,CBSA#,stab,CBSA,Year
0,1,51,0,0,149000,1,23,0,9,3,...,43.0,1.0,22.0,3.0,1.0,1.0,35620.0,NY,"New York-Newark-Jersey City, NY-NJ-PA",2012
1,4,104,0,0,50000,1,33,0,1,2,...,56.0,1.0,22.0,1.0,1.0,1.0,26420.0,TX,"Houston-The Woodlands-Sugar Land, TX",2012
2,7,94,0,0,5000,2,101,0,1,3,...,75.0,1.0,13.0,1.0,6.0,1.0,37980.0,PA,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",2012
3,16,90,0,580,0,3,0,0,5,9,...,57.0,2.0,16.0,1.0,6.0,1.0,19820.0,MI,"Detroit-Warren-Dearborn, MI",2012
4,19,180,0,0,-60000,0,0,4,6,9,...,NaN,NaN,NaN,NaN,NaN,NaN,41860.0,CA,"San Francisco-Oakland-Berkeley, CA",2012


### Add Area Median Income (AMI) Values for each CBSA/MSA for the years 2012 - 2019
### Recalculate Values as HINCP (Household Income past 12 months)  / AMI

In [9]:
# Get the Area Median Income (AMI) values for each MSA for each year of the total acs dataset

ami = pd.read_excel('AMI50MSA.xlsx', header = 1)
ami.drop(ami.columns[[10,11,12,13]], axis=1, inplace=True)
ami.set_index('CBSA', inplace=True)
ami

,2012,2013,2014,2015,2016,2017,2018,2019,2021
CBSA,,,,,,,,,
"Atlanta-Sandy Springs-Alpharetta, GA",54628,55733,56166,60219,62613,65381,69464,71742,77589
"Austin-Round Rock-Georgetown, TX",59433,61750,63603,67195,71000,73800,76925,80954,86530
"Baltimore-Columbia-Towson, MD",66970,68455,71501,72520,76788,77394,80469,83160,86302
"Birmingham-Hoover, AL",46763,48328,47046,51459,52226,53107,56409,58366,60725
"Boston-Cambridge-Newton, MA-NH",71738,72907,75667,78800,82380,85691,88711,94430,100750
"Buffalo-Cheektowaga, NY",50269,50548,50074,51772,53487,55448,56195,60105,62794
"Charlotte-Concord-Gastonia, NC-SC",52470,51251,53549,54836,59979,61156,62068,66399,71041
"Chicago-Naperville-Elgin, IL-IN-WI",59261,60564,61598,63153,66020,68403,70760,75379,78166
"Cincinnati, OH-KY-IN",52439,53378,55729,56826,60260,61653,62743,66825,70818


In [10]:
# Force all HINCP (Household income past 12 months) to numeric values in Merged ACS dataset

acsTotal.HINCP = pd.to_numeric(acsTotal.HINCP, errors='coerce')

In [11]:
# Create dataset with empty values for AMI values each year

acsAMI = acsTotal[['SERIALNO','HINCP','CBSA','PUMA','Year']]
acsAMI['AMI_CAT_2012'] = np.NaN
acsAMI['AMI_CAT_2013'] = np.NaN
acsAMI['AMI_CAT_2014'] = np.NaN
acsAMI['AMI_CAT_2015'] = np.NaN
acsAMI['AMI_CAT_2016'] = np.NaN
acsAMI['AMI_CAT_2017'] = np.NaN
acsAMI['AMI_CAT_2018'] = np.NaN
acsAMI['AMI_CAT_2019'] = np.NaN
#acsAMI['AMI_CAT_2021'] = np.NaN

print('\033[1m' + 'AMI Category DataFrame')
print('{} Features, {} Entries'.format(acsAMI.shape[1], acsAMI.shape[0]))
acsAMI.head()

AMI Category DataFrame
13 Features, 5827854 Entries


,SERIALNO,HINCP,CBSA,PUMA,Year,AMI_CAT_2012,AMI_CAT_2013,AMI_CAT_2014,AMI_CAT_2015,AMI_CAT_2016,AMI_CAT_2017,AMI_CAT_2018,AMI_CAT_2019
0,1,149000,"New York-Newark-Jersey City, NY-NJ-PA",3106,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,50000,"Houston-The Woodlands-Sugar Land, TX",4601,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,5000,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",3211,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,0,"Detroit-Warren-Dearborn, MI",2907,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,-60000,"San Francisco-Oakland-Berkeley, CA",7502,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Calculate HINCP/AMI for correspondings MSAs and years
# Takes some time (10 min)
# Always restart process from loading AMI file first

for x in range(acsAMI.shape[0]):
  #print('ON ROW:',x)
  if (acsAMI.loc[x,'Year']) ==2012:
    cbsa = acsAMI.loc[x, 'CBSA']
    acsAMI.loc[x,'AMI_CAT_2012'] = acsAMI.loc[x,'HINCP']/ ami.loc[cbsa,2012]
  elif (acsAMI.loc[x,'Year']) ==2013:
    cbsa = acsAMI.loc[x, 'CBSA']
    acsAMI.loc[x,'AMI_CAT_2013'] = acsAMI.loc[x,'HINCP']/ ami.loc[cbsa,2013]
  elif (acsAMI.loc[x,'Year']) ==2014:
    cbsa = acsAMI.loc[x, 'CBSA']
    acsAMI.loc[x,'AMI_CAT_2014'] = acsAMI.loc[x,'HINCP']/ ami.loc[cbsa,2014]
  elif (acsAMI.loc[x,'Year']) ==2015:
    cbsa = acsAMI.loc[x, 'CBSA']
    acsAMI.loc[x,'AMI_CAT_2015'] = acsAMI.loc[x,'HINCP']/ ami.loc[cbsa,2015]
  elif (acsAMI.loc[x,'Year']) ==2016:
    cbsa = acsAMI.loc[x, 'CBSA']
    acsAMI.loc[x,'AMI_CAT_2016'] = acsAMI.loc[x,'HINCP']/ ami.loc[cbsa,2016]
  elif (acsAMI.loc[x,'Year']) ==2017:
    cbsa = acsAMI.loc[x, 'CBSA']
    acsAMI.loc[x,'AMI_CAT_2017'] = acsAMI.loc[x,'HINCP']/ ami.loc[cbsa,2017]
  elif (acsAMI.loc[x,'Year']) ==2018:
    cbsa = acsAMI.loc[x, 'CBSA']
    acsAMI.loc[x,'AMI_CAT_2018'] = acsAMI.loc[x,'HINCP']/ ami.loc[cbsa,2018]
  elif (acsAMI.loc[x,'Year']) ==2019:
    cbsa = acsAMI.loc[x, 'CBSA']
    acsAMI.loc[x,'AMI_CAT_2019'] = acsAMI.loc[x,'HINCP']/ ami.loc[cbsa,2019]
  #elif (acsAMI.loc[x,'Year']) ==2021:
  #  cbsa = acsAMI.loc[x, 'CBSA']
  #  acsAMI.loc[x,'AMI_CAT_2021'] = acsAMI.loc[x,'HINCP']/ ami.loc[cbsa,2021]


In [13]:
# Double check dataset ... AMI_CAT_2012 should show first five values, AMI_CAT_2019 should show last five values

acsAMI

,SERIALNO,HINCP,CBSA,PUMA,Year,AMI_CAT_2012,AMI_CAT_2013,AMI_CAT_2014,AMI_CAT_2015,AMI_CAT_2016,AMI_CAT_2017,AMI_CAT_2018,AMI_CAT_2019
0,1,149000,"New York-Newark-Jersey City, NY-NJ-PA",3106,2012,2.328780,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,50000,"Houston-The Woodlands-Sugar Land, TX",4601,2012,0.894294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,5000,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",3211,2012,0.083188,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,0,"Detroit-Warren-Dearborn, MI",2907,2012,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,-60000,"San Francisco-Oakland-Berkeley, CA",7502,2012,-0.800833,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5827849,2019HU1412409,220000,"San Jose-Sunnyvale-Santa Clara, CA",8511,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.681122
5827850,2019HU1412415,73500,"Chicago-Naperville-Elgin, IL-IN-WI",3005,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.975073
5827851,2019HU1412427,76900,"Portland-Vancouver-Hillsboro, OR-WA",1314,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.980380
5827852,2019HU1412433,90000,"Dallas-Fort Worth-Arlington, TX",2509,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.245416


In [14]:
# Merge total acs dataset with AMI categories

acsFinal = pd.merge(acsTotal,acsAMI,on = ['SERIALNO', 'PUMA', 'CBSA', 'Year', 'HINCP'], how = 'left')
print('\033[1m' + 'ACS + AMI dataset')
print('{} Features, {} Entries'.format(acsFinal.shape[1], acsFinal.shape[0]))
acsFinal.head()

ACS + AMI dataset
30 Features, 5827854 Entries


,SERIALNO,WGTP,GRPIP,RNTP,HINCP,TEN,OCPIP,VACS,YBL,BLD,...,CBSA,Year,AMI_CAT_2012,AMI_CAT_2013,AMI_CAT_2014,AMI_CAT_2015,AMI_CAT_2016,AMI_CAT_2017,AMI_CAT_2018,AMI_CAT_2019
0,1,51,0,0,149000,1,23,0,9,3,...,"New York-Newark-Jersey City, NY-NJ-PA",2012,2.328780,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,104,0,0,50000,1,33,0,1,2,...,"Houston-The Woodlands-Sugar Land, TX",2012,0.894294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,94,0,0,5000,2,101,0,1,3,...,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",2012,0.083188,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,90,0,580,0,3,0,0,5,9,...,"Detroit-Warren-Dearborn, MI",2012,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,180,0,0,-60000,0,0,4,6,9,...,"San Francisco-Oakland-Berkeley, CA",2012,-0.800833,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Perform Data Transformations / Create New Features (Done in SPSS Syntax_50MSA.sps)

In [15]:
# Perform transformations on dataset (Replicating previous SPSS processing)

def transformData(df):
    
    # Bin AMI_CAT variables
    # (Min-.3] = 1, (.3-.5] = 2, (.5-.8] = 3, (.8-1.2] = 4, (1.2-2.0] = 5, (2.0-Max] = 6
    for year in range(2012, 2020):
        ami = 'AMI_CAT_' + str(year)
        df[ami] = pd.cut(df[ami], bins = [-np.inf, 0.3, 0.5, 0.8, 1.2, 2.0, np.inf], 
                                    labels = [1, 2, 3, 4, 5, 6])
        
    # Bin TEN (tenure) variable into two categories: owner (1) and renter (2)
    df['Own_Rent'] = pd.cut(df['TEN'], bins = [-1, 0.5, 2, 4], labels = [0, 1, 2])
    
    # Bin AGEP variable into 8 bins (0 - 7)
    # (Min-15] = 0, (15-25] = 1, (25-35] = 2, (35-45] = 3, (45-55] = 4, (55-65] = 5, (65-75] = 6, (75-Max] = 7
    df['Age_Cat'] = pd.cut(df['AGEP'], bins = [-np.inf, 14.5, 24.5, 34.5, 44.5, 54.5, 64.5, 74.5, np.inf], 
                            labels = [0, 1, 2, 3, 4, 5, 6, 7])
    
    
    # Bin VACS variable into 2 bins - Vacant (0) and Not Vacant (1)
    df['VAC_Flag'] = pd.cut(df['VACS'], bins = [-1, 0.5, 8], labels = [0, 1])
    
    # Bin BLD (# of Units in Structure) variable into 4 bins - 1Unit, 2-4Units, 5-49Units, 50+Units
    # 1(MobileHomes/Trailers) = 0, 2-3(1Unit) = 1, 4-5(2-4Units) = 2, 6-8(5-49Unitsrf) = 3, 9(50+Units) = 4
    df['BLD_TYPE'] = pd.cut(df['BLD'], bins = [-1, 1.5, 3.5, 5.5, 8.5, 9.5, 11], 
                            labels = [0, 1, 2, 3, 4, 0], ordered=False)
    
    # Calculate Rent Burden
    # Bin GRPIP (Gross Rent as Percentage of Household Income Past 12 Months) into 3 Categories as 'rent_burden'
    # [1-29] = 1, (30-49] = 2, (50+] = 3
    df['rent_burden'] = pd.cut(df['GRPIP'], bins = [-0.5, 0.5, 29.5, 49.5, np.inf], 
                               labels = [0, 1, 2, 3])
    # Missing and zero rent payments are classified as 'no burden'
    # Bin RNTP (Monthy Rent per Month) to new variable 'No_RNTP': 0 = 1, All other entries = 0
    df['No_RNTP'] = pd.cut(df['RNTP'], bins = [-1, 0.5, np.inf], 
                           labels = [1, 0], ordered = False)
    # Households with negative income that pay rent are classified as 'severe burden'
    # Bin HINCP (Household Income Past 12 Months) to new variable 'No_Inc': Min - 0 = 3, All other entries = 0
    df['No_Inc'] = pd.cut(df['HINCP'], bins = [-np.inf, 0.1, np.inf], 
                          labels = [3, 0], ordered = False)
    # Sum 'rent_burden', 'No_RNTP', and 'No_Inc' as a single variable:'Rent_Burd_Cat'
    # If either 'No_RNTP' or 'No_Inc' is not 0, 'rent_burden' will be 0 
    # Convert 'Rent_Burd_Cat' value 4 to 3
    df['Rent_Burd_Cat'] = df['rent_burden'].astype('int64') + df['No_RNTP'].astype('int64') + df['No_Inc'].astype('int64')
    df.loc[df['Rent_Burd_Cat'] == 4, 'Rent_Burd_Cat'] = 3

    # Bin 'VACS' (vacancy status): 1-2 = 1, All other entries = 0
    df['Vac_Rent'] = pd.cut(df['VACS'], bins = [-np.inf, 0.9, 2.1, np.inf], 
                               labels = [0, 1, 0], ordered = False)
    
    # Create categorical household income variable
    # Bin 'HINCP' to new variable 'hhinc_cat' as 6 categories
    # Min-19999 = 1, 20000-34999 = 2, 35000-49999 = 3, 50000-74999 = 4, 75000-Max = 5, All other entries = 6
    df['hhinc_cat'] = pd.cut(df['HINCP'], bins = [-np.inf, 19999.5, 34999.5, 49999.5, 74999.5, np.inf], 
                               labels = [1, 2, 3, 4, 5], ordered = False)
    
    # Bin 'OCPIP' (Selected monthly owner costs as a percentage of household income during the past 12 months) 
    # Create variable 'owner_burden' for owner cost burden: Min-29 = 1, 30-Max = 2
    df['owner_burden'] = pd.cut(df['OCPIP'], bins = [-np.inf, 29.5, np.inf], labels = [1, 2])

    # Create a flag 'Rent_Flag' to flag renters: if'Own_Rent' is 2, flag as 1
    df.loc[df['Own_Rent'] == 2, 'Rent_Flag'] = 1
    
    # Bin 'YBL' (Year Building was built): 01-02 = 1pre1950, 08-Max = 2post2000, All other entries = 0Other
    df['Age_Old_New'] = pd.cut(df['YBL'], bins = [0.5, 2.5, 7.5, np.inf], 
                                labels = ['1pre1950', '0Other', '2post2000'], ordered=False)
    df['Age_Old_New'] = df['Age_Old_New'].fillna('0Other')
    
    # Bin 'HISP' (detailed hispanic origin) as 'LatinoOrg': 1 = '1NonHispanic', [2-25] = '2Hispanic'
    # NaN values not converted to 'ONA' like SPSS
    df['LatinoOrg'] = pd.cut(df['HISP'].fillna(np.nan), bins = [0.5, 1.5, float(np.inf)], 
                                labels = ['1NonHispanic', '2Hispanic'], ordered=False)
    
    # Bin 'RAC1P' (race code) to 'Race_Recoded'
    # NaN values not converted to 'ONA' like SPSS
    # 1 = 'White', 2 = '2Black', 6 = '3Asian', 3-5 = '4Other', 7-8 = '4Other', 9 = '5TwoMore'
    df['Race_Recoded'] = pd.cut(df['RAC1P'].fillna(np.nan), bins = [0.5, 1.5, 2.5, 5.5, 6.5, 8.5, 9.5], 
                                labels = ['1White', '2Black', '4Other', '3Asian', '4Other', '5TwoMore'], ordered=False)
    
    # Bin 'ESR' (employment status code) to new variable 'Unemployment_status'
    # NaN values not converted to 'ONA' like SPSS
    # 1-2 = '0Employed', 3 = '1Umemployed', 4-5 = '0Employed', 6 = '2NotInLaborForce'
    df['Unemployment_status'] = pd.cut(df['ESR'].fillna(np.nan), bins = [0.5, 2.5, 3.5, 5.5, 6.5], 
                                labels = ['0Employed', '1Unumployed', '0Employed', '2NotInLaborForce'], ordered=False)
    
    # Bin 'SCHL' (educational attainment) to new variable 'Ed_status'
    # 1-15 = '1NoHSdiploma', 21-Max = '2BachDegreeAndMore', All other entries = '3Other'
    df['Ed_status'] = pd.cut(df['SCHL'].fillna(np.nan), bins = [0.5, 15.5, 20.5, 24.5], 
                                labels = ['1NoHSdiploma', '3Other', '2BachDegreeAndMore'], ordered=False)
    df['Ed_status'] = df['Ed_status'].fillna('3Other')
    
    # Create a flag for mobile homes
    # Change 'BLD' to new variable 'MOBILE_Bld_type': 1 = 1, All other entries = 0
    df['MOBILE_Bld_type'] = pd.cut(df['BLD'].fillna(np.nan), bins = [-0.5, 0.5, 1.5, 10.5], 
                                labels = [0, 1, 0], ordered=False)
    
    
    return df

In [16]:
# Perform all transformations done in SPSS

acsFinalTF = transformData(acsFinal)

print('\033[1m' + 'ACS + AMI Dataset After Data Transformations')
print('{} Features, {} Entries'.format(acsFinalTF.shape[1], acsFinalTF.shape[0]))
acsFinalTF


ACS + AMI Dataset After Data Transformations
48 Features, 5827854 Entries


,SERIALNO,WGTP,GRPIP,RNTP,HINCP,TEN,OCPIP,VACS,YBL,BLD,...,Vac_Rent,hhinc_cat,owner_burden,Rent_Flag,Age_Old_New,LatinoOrg,Race_Recoded,Unemployment_status,Ed_status,MOBILE_Bld_type
0,1,51,0,0,149000,1,23,0,9,3,...,0,5,1,NaN,2post2000,2Hispanic,1White,0Employed,2BachDegreeAndMore,0
1,4,104,0,0,50000,1,33,0,1,2,...,0,4,2,NaN,1pre1950,1NonHispanic,1White,0Employed,2BachDegreeAndMore,0
2,7,94,0,0,5000,2,101,0,1,3,...,0,1,2,NaN,1pre1950,1NonHispanic,1White,2NotInLaborForce,1NoHSdiploma,0
3,16,90,0,580,0,3,0,0,5,9,...,0,1,1,1.0,0Other,1NonHispanic,2Black,2NotInLaborForce,3Other,0
4,19,180,0,0,-60000,0,0,4,6,9,...,0,1,1,NaN,0Other,NaN,NaN,NaN,3Other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5827849,2019HU1412409,146,14,2500,220000,3,0,0,6,6,...,0,5,1,1.0,0Other,1NonHispanic,1White,0Employed,2BachDegreeAndMore,0
5827850,2019HU1412415,61,0,0,73500,1,26,0,6,2,...,0,4,1,NaN,0Other,1NonHispanic,1White,0Employed,3Other,0
5827851,2019HU1412427,61,0,0,76900,2,13,0,1,2,...,0,5,1,NaN,1pre1950,1NonHispanic,1White,2NotInLaborForce,2BachDegreeAndMore,0
5827852,2019HU1412433,282,22,1400,90000,3,0,0,5,7,...,0,5,1,1.0,0Other,1NonHispanic,2Black,0Employed,3Other,0


In [17]:
acsFinalTF

,SERIALNO,WGTP,GRPIP,RNTP,HINCP,TEN,OCPIP,VACS,YBL,BLD,...,Vac_Rent,hhinc_cat,owner_burden,Rent_Flag,Age_Old_New,LatinoOrg,Race_Recoded,Unemployment_status,Ed_status,MOBILE_Bld_type
0,1,51,0,0,149000,1,23,0,9,3,...,0,5,1,NaN,2post2000,2Hispanic,1White,0Employed,2BachDegreeAndMore,0
1,4,104,0,0,50000,1,33,0,1,2,...,0,4,2,NaN,1pre1950,1NonHispanic,1White,0Employed,2BachDegreeAndMore,0
2,7,94,0,0,5000,2,101,0,1,3,...,0,1,2,NaN,1pre1950,1NonHispanic,1White,2NotInLaborForce,1NoHSdiploma,0
3,16,90,0,580,0,3,0,0,5,9,...,0,1,1,1.0,0Other,1NonHispanic,2Black,2NotInLaborForce,3Other,0
4,19,180,0,0,-60000,0,0,4,6,9,...,0,1,1,NaN,0Other,NaN,NaN,NaN,3Other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5827849,2019HU1412409,146,14,2500,220000,3,0,0,6,6,...,0,5,1,1.0,0Other,1NonHispanic,1White,0Employed,2BachDegreeAndMore,0
5827850,2019HU1412415,61,0,0,73500,1,26,0,6,2,...,0,4,1,NaN,0Other,1NonHispanic,1White,0Employed,3Other,0
5827851,2019HU1412427,61,0,0,76900,2,13,0,1,2,...,0,5,1,NaN,1pre1950,1NonHispanic,1White,2NotInLaborForce,2BachDegreeAndMore,0
5827852,2019HU1412433,282,22,1400,90000,3,0,0,5,7,...,0,5,1,1.0,0Other,1NonHispanic,2Black,0Employed,3Other,0


In [18]:
a = pd.read_csv(r"C:\Users\Stephen\Desktop\Projects\IHS\acsProccessed.csv")
a

,V1,SERIALNO,WGTP,GRPIP,RNTP,HINCP,TEN,OCPIP,VACS,YBL,...,Vac_Rent,hhinc_cat,owner_burden,Rent_Flag,Age_Old_New,LatinoOrg,Race_Recoded,Unemployment_status,Ed_status,MOBILE_Bld_type
0,4,19,180,0,0,-60000,0,0,4,6,...,0,1,1,,0Other,0NA,NaN,NaN,3Other,0
1,7,43,214,0,470,-60000,0,0,2,1,...,1,1,1,,1pre1950,0NA,NaN,NaN,3Other,0
2,14,76,0,0,0,-60000,0,0,0,0,...,0,1,1,,0Other,2Hispanic,4Other,2NotinLaborForce,3Other,0
3,18,100,0,0,0,-60000,0,0,0,0,...,0,1,1,,0Other,1NonHispanic,1White,0Employed,3Other,0
4,22,137,0,0,0,-60000,0,0,0,0,...,0,1,1,,0Other,1NonHispanic,1White,2NotinLaborForce,3Other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5827849,4686490,2018HU0507413,114,1,1900,2580000,3,0,0,20,...,0,5,1,1,2post2000,1NonHispanic,1White,0Employed,2BachDegreeAndMore,0
5827850,5181803,2019HU0501047,156,0,0,2868000,1,3,0,4,...,0,5,1,,0Other,2Hispanic,1White,0Employed,2BachDegreeAndMore,0
5827851,5218282,2019HU0727228,113,0,0,2907600,1,1,0,7,...,0,5,1,,0Other,1NonHispanic,1White,0Employed,3Other,0
5827852,3201027,587402,76,0,0,3164000,2,1,0,1,...,0,5,1,,1pre1950,1NonHispanic,2Black,2NotinLaborForce,1NoHSdiploma,0
